# Import modules

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time
import pickle
import gc
from scipy import sparse
from math import sqrt
from numpy import random as rd

# ML libraries
from sklearn.decomposition import NMF
from sklearn.utils.extmath import randomized_svd
from sklearn.metrics import mean_squared_error

# Set notation of values 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set directory - where the data is located

In [2]:
_dir = "D:\Python\Thesis\cf_ready_data\\"
os.chdir(_dir)

# Define metrics

In [3]:
# To compare both models we need a metric to unify the algorithms. We use RMSE since this approach provides a nice
# meaning in the metric. i.e avg difference between what rating we get vs what we predicted

def calc_rmse(actual, pred):
    """
    Inputs: actual, a numpy array of our observed data
            pred, a numpy array of our predicted data, for NMF it would be W*H
    Output: rmse
    """
    n = actual.sum() # this is we only want to sum entries that exist
    se = (actual - pred)**2
    
    return sqrt((1/n)*se.sum())

In [4]:
# prediction function
# there are two ways to do prediction: 
# 1) look at entries of WH on a specific row 
# 2) np.dot(W[i,:],H[i,:]

# point 2 is for comparing other contracts and recommend clauses from them via nearest neighbours

def predict(X_hat, actual, ref, filter_actual, idx, n_rec):
    """
    Inputs: X_hat : numpy array, in NMF this is WH, in SVD this is USV
            actual: a numpy array, the original X matrix
            ref: a pd.DataFrame object, a DataFrame version of X_hat
            filter_actual: a boolean value, chooses to get rid of actuals
            idx: an integer, contract index
            n_rec: an integer, shows top n_rec rows
            
    Output: pandas DataFrame object, subsetted by idx
    
    Notes: this function relies on ratings_matrix(), and new_dict3 which is the finalised processed data
    new_dict3 = tfidf_summarisation(clauses_list, new_dict2, 10)
    
    """
    # Create a df for reference to obtain actual clauses
    # Idea is to append this to the table so that we preserve clause 
    #df_to_get_clauses = ratings_matrix(new_dict3, to_df = True, transpose = False, fill_val = 0)
    #df_toget_clauses = pd.read_pickle(ref)
    df_to_get_clauses = ref
    clauses = df_to_get_clauses.columns.values
    #  Condition to remove actuals and leave preds
    if filter_actual == True:
        
        diff = X_hat - actual
        diff = np.clip(diff,0,1)
        
    else:
        
        diff = X_hat
        
    print("For contract", idx, "the top", n_rec, "recommended clauses are:")
    
    # return the data set subsetted on row by some idx, and sort descending and show n_rec of them
    return pd.DataFrame(data=diff[idx,:], index = clauses).sort_values(by=0, axis = 0, ascending = False).head(n_rec)

# Loading the Ratings Matrix

In [5]:
file1 = open("X3_df.pickle", "rb")
X_df = pickle.load(file1)

file2 = open("X3.pickle", "rb")
X = pickle.load(file2)
X.shape

(1479, 14427)

# NMF

Here we employ NMF: Non-negative matrix factorisation.

Our goal in NMF is to approximate this matrix by the dot product of two arrays $W$ and $H$. 

Dimensions of the arrays are defined by dimensions of $X$ and number of components we set to the algorithm. If $X$ has $n$ contracts/rows and $m$ clauses/columns and we want to decompose it to $k$ clauses/columns, then $W$ has $n$ contracts/rows, and $k$ clauses/rows and $H$ has $k$ clauses/rows and $m$ contracts/columns.

$X$ is our contract-clauses matrix of dimension $n \times m$ i.e contracts = rows, clauses = cols

$W$ is interpreted as if a contract has clause $y$, what is the additional assignment weight to a group or in our case "similar-clauses"

$H$ The higher the weight value the more the clause belonging to a group of "similar-clauses".

Both W,H are initialised as some value - similar to how in NN's weights and biases have an initialisation.

Good example and interpretation: https://medium.com/logicai/non-negative-matrix-factorization-for-recommendation-systems-985ca8d5c16c

In [6]:
# Recall that NMF seeks to break down a matrix X into W and H
# Such that X ≈ W*H

def train_val_NMF_model(data, components, alph, cross_val, method, export, verbose):
    """
    Inputs: data: numpy array object, for fit_transform() method
            components: a list object, range of component parameters
            alph: a list object, range of regularisation parameters
            cross_val: a int object, defines number of k-fold cross val
            method: a string object, defines what initialisation is needed for NMF training
            export: a boolean value, exports metrics to a dictionary
            verbose: a boolean value, turns on verbose on or off
            
    Outputs: errors: a list of frobenius norm of residual matrix between data and the representation(W,H)
             config: a list of configurations used to get the errors
             Ws: a list of W components for each configuration of [components, alph] s.t X ≈ W*H
             Hs: a list of H components for each configuration of [components, alph] s.t X ≈ W*H
    """
    if type(verbose) != bool:
        raise ValueError("'verbose' variable is not boolean-type, use 'True' or 'False' to control verbose")
    else:
        pass
    
    
    o_start = time.perf_counter()
    errors = []
    cv_errors = []
    config = []
    Ws = []
    Hs = []
    
    print("Initialisation:", method)
    print("Training and validating...")
    for comp in components:
        start = time.perf_counter()
        for alphas in alph:
            print("Config (components, alpha):",[comp,alphas])
            for cv in range(cross_val):
                np.random.shuffle(data) # shuffle data to perform cv and get rmse
                NMF_model = NMF(
                            verbose = verbose,
                            n_components = comp,
                            init = method, 
                            solver = 'mu',
                            beta_loss = 'frobenius', # also called Euclidean Norm
                            tol = 1e-4,
                            random_state = 0,
                            alpha = alphas,
                            max_iter = 100
                           )
            
                if verbose == True:
                    print("Computing W...")
                else:
                    pass
    
                W = NMF_model.fit_transform(data)
        
                if verbose == True:
                    print("Computing H...")
                else:
                    pass
            
                H = NMF_model.components_

                cv_error = calc_rmse(X, (W@H))
                #error = sqrt(error.mean())
                print(cv,"-fold" ,"train_rmse:", cv_error)
                cv_errors.append(cv_error)
                
            print("Avg rmse:", np.mean(cv_error)) 
            print("Training time elapsed in minutes: ", (time.perf_counter() - start)/60) 
            print("")
            errors.append(np.mean(cv_error))
            config.append([comp, alphas])
            Ws.append(sparse.csr_matrix(W))
            Hs.append(sparse.csr_matrix(H))
            gc.collect()
    
    argmin_error = np.argmin(errors)
    best_config = config[argmin_error]
    best_error = errors[argmin_error]
    
    if export == True:
        metrics = {"best_train_config": best_config,
                   "best_train_rmse": best_error,
                   "train_config": config,
                   "train_rmse": errors,
                   "Ws": Ws,
                   "Hs": Hs      
                  }
        out = open("D:\Python\Thesis\metrics\\" + "nmf_" + 'metrics.pickle' ,"wb")
        pickle.dump(metrics, out)
        out.close
    else:
        pass
    
    print("Training and validating complete")        
    print("Total time elapsed in minutes: ", (time.perf_counter() - o_start)/60) 
    print("")
    print("Best configuration:", best_config, "with error:", best_error)
    print("Subset W, H at index:", argmin_error)
    print("---------------------------------------")
    return errors, config, Ws, Hs

In [ ]:
comp = [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400]
#comp = [300]
r_error, r_configs, r_Ws, r_Hs = train_val_NMF_model(data = X,
                                                     components = comp, 
                                                     alph = [0.01], 
                                                     cross_val = 5,
                                                     method = 'nndsvdar',
                                                     export = True,
                                                     verbose = False)

Initialisation: nndsvdar
Training and validating...
Config (components, alpha): [100, 0.01]
0 -fold train_rmse: 0.8162168782884001
1 -fold train_rmse: 0.8161847382636858
2 -fold train_rmse: 0.8160648100509018
3 -fold train_rmse: 0.8159249892409699
4 -fold train_rmse: 0.817024643043419
Avg rmse: 0.817024643043419
Training time elapsed in minutes:  0.6121474500499999

Config (components, alpha): [200, 0.01]
0 -fold train_rmse: 0.7141081205834762
1 -fold train_rmse: 0.7140116166454343
2 -fold train_rmse: 0.7139630770884106
3 -fold train_rmse: 0.7145120248939532
4 -fold train_rmse: 0.7138248536071132
Avg rmse: 0.7138248536071132
Training time elapsed in minutes:  1.2418213757499998

Config (components, alpha): [300, 0.01]
0 -fold train_rmse: 0.6373105998997659
1 -fold train_rmse: 0.6380719644330592
2 -fold train_rmse: 0.6368708531298146
3 -fold train_rmse: 0.6373709813106713
4 -fold train_rmse: 0.6376457437282267
Avg rmse: 0.6376457437282267
Training time elapsed in minutes:  2.06601687368

d:\program files\pycharm 2019.2\venv\lib\site-packages\sklearn\decomposition\nmf.py:1069: ConvergenceWarning: Maximum number of iteration 100 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


1 -fold train_rmse: 0.5100578317413397
2 -fold train_rmse: 0.5102032222798351
3 -fold train_rmse: 0.5111885077766113
4 -fold train_rmse: 0.5104031106876948
Avg rmse: 0.5104031106876948
Training time elapsed in minutes:  4.351201239883332

Config (components, alpha): [600, 0.01]
0 -fold train_rmse: 0.45299057479683197
1 -fold train_rmse: 0.4527254713764287


## NMF predictions

In [ ]:
NMF_Xhat = r_Ws[14].todense()@r_Hs[14].todense()
#pd.DataFrame(NMF_Xhat)

# SVD

Core idea of SVD is similar to NMF where we want to express our contract-clauses matrix as a product of matrices in a smaller dimension.

The only difference is the training process and the components we obtain. In NMF we obtain two components $W,H$ and in SVD we obtain three components $U,S,V^T$. SVD components are obtained via linear algebra techniques.

But there is very little interpretability - hard to explain to non-technical people what is going on.

In addition to looking at the entries for predictions, SVD allows approach allows us to project a specific contract into a smaller space and thus compare contracts (via some distance metric) and get recommendations from similar contracts.

In [ ]:
# Here we use SVD approach, idea is we want to decompose X = UΣV^(T)

def train_val_SVD_model(data, components, cross_val, export):
    """
    Inputs: data: numpy array object, for fit_transform() method
            components: a list object, range of component parameters
            cross_val: a int object, defines number of k-fold cross val
            export: a boolean value, exports metrics
            
    Outputs: errors: a list of frobenius norm of residual matrix between data and the representation(W,H)
             config: a list of configurations used to get the errors

    """
    start = time.perf_counter()
    config = []
    U_list = []
    S_list = []
    V_t_list = []
    cv_rmse_list = []
    avg_rmse_list = []
    
    print("Performing tSVD...")
    for comp in components:
        print("Number of components:", comp)
        for cv in range(cross_val):
            np.random.shuffle(data) # shuffle data to perform cv and get rmse
            U, S_placeholder, V_t = randomized_svd(
                        M = data,
                        n_components = comp,
                        random_state = 0
                        )
            # sklearn returns a list of components, but it should be in a matrix where these values are in diagonal entries
            S = np.ndarray(shape = (S_placeholder.shape[0],S_placeholder.shape[0]))
            np.fill_diagonal(S, S_placeholder)

            # Reconstruction of the data
            data_pred = U@S@V_t

            rmse = calc_rmse(data, data_pred)
        
            print(cv,"fold","RMSE:", rmse)
            print("")
        
            cv_rmse_list.append(rmse)
        
        print("Avg rmse:", np.mean(cv_rmse_list))
        avg_rmse_list.append(np.mean(cv_rmse_list))
        config.append([comp])
        U_list.append(sparse.csr_matrix(U))
        S_list.append(sparse.csr_matrix(S))
        V_t_list.append(sparse.csr_matrix(V_t))
    
    lowest_rmse_idx = np.argmin(avg_rmse_list)
    
    if export == True:
        metrics = {"best_idx": lowest_rmse_idx,
                   "best_config": config[lowest_rmse_idx],
                   "best_U": U_list[lowest_rmse_idx],
                   "best_S": S_list[lowest_rmse_idx],
                   "best_Vt":V_t_list[lowest_rmse_idx]
                  }
        out = open("D:\Python\Thesis\metrics\\" + "svd_" + '_metrics.pickle' ,"wb")
        pickle.dump(metrics, out)
        out.close
    else:
        pass
    
    print("tSVD complete") 
    print("Best # of components to choose is", config[lowest_rmse_idx],",","Subset on index:",lowest_rmse_idx)
    print("Time elapsed in minutes: ", (time.perf_counter() - start)/60) 
    
    return config, U_list, S_list, V_t_list

In [ ]:
conf, U, S, V = train_val_SVD_model(X, comp, export = True)

## SVD predictions

In [ ]:
#SVD_Xhat = U[]@S[]@V[]

In [ ]:
#predict(SVD_Xhat, X, True, 21, 10) 